<a href="https://colab.research.google.com/github/GiPaiva/Algoritmo-Genetico/blob/main/EP2_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Integrantes: Giovanna Paiva Alves e Matheus Sanchez Duda

# Equipamentos no Laboratório de Química do SENAC

Suponha que um laboratório de química precisa organizar o uso de seus equipamentos para garantir que todas as análises necessárias possam ser realizadas dentro do prazo esperado e que as restrições de segurança e capacidade dos equipamentos sejam cumpridas. O laboratório possui um conjunto de equipamentos, como balanças, espectrômetros e microscópios, que são usados para realizar diferentes tipos de análises químicas.

Existem várias análises que precisam ser realizadas, cada uma com seus próprios requisitos de equipamentos e tempo de execução. Alguns equipamentos só podem ser usados para uma análise específica, enquanto outros podem ser compartilhados entre diferentes análises.

As restrições a serem satisfeitas incluem:

1. Cada equipamento tem uma capacidade máxima de uso diário e só pode ser usado para uma análise por vez.
2. Uma análise não pode estar em 2 equipamentos ao mesmo tempo.
3. Cada análise fica 1 hora em cada equipamento

O objetivo é encontrar um cronograma de uso de equipamentos que atenda a todas as restrições e minimize o tempo total necessário para concluir todas as análises.


In [5]:
import random
from collections import defaultdict
import copy

class Individuo:
    def __init__(self, analyses, equipment_limits):
        # Analyses and required equipment
        self.analyses = analyses

        # Equipment and their daily usage limits (in hours)
        self.equipment_limits = equipment_limits

        # Schedule representation: day -> hour -> [(equipment, analysis)]
        self.schedule = defaultdict(lambda: defaultdict(list))

        # Initialize equipment usage tracking: day -> equipment -> hours used
        self.equipment_usage = defaultdict(lambda: defaultdict(int))

        # Schedule validity flag
        self.is_schedule_valid = False

        # Initialize with a random schedule
        self.initialize_random_schedule()

    def initialize_random_schedule(self):
        """Initialize a random schedule for all analyses."""
        # Create a list of all possible day-hour slots (7 days, hours de 8 às 17)
        all_slots = [(day, hour) for day in range(1, 8) for hour in range(8, 18)]

        # Shuffle the list de análises para randomização
        analyses_list = list(self.analyses.keys())
        random.shuffle(analyses_list)

        # Tenta agendar cada análise
        for analysis in analyses_list:
            equipment_list = self.analyses[analysis]

            # Para cada equipamento necessário pela análise
            for equipment in equipment_list:
                # Cria uma cópia dos slots para tentar alocar
                available_slots = all_slots.copy()
                random.shuffle(available_slots)

                scheduled = False
                for day, hour in available_slots:
                    # Verifica se o slot é válido
                    if self.is_valid_timeslot(analysis, day, hour, equipment):
                        self.schedule[day][hour].append((equipment, analysis))
                        self.equipment_usage[day][equipment] += 1
                        scheduled = True
                        break

                # Se não conseguir agendar este equipamento, o indivíduo fica inválido
                if not scheduled:
                    self.is_schedule_valid = False
                    return

        # Se todas as análises foram agendadas, marca o indivíduo como válido
        self.is_schedule_valid = True

    def is_valid_timeslot(self, analysis, day, hour, equipment):
        """Verifica se o slot (dia, hora) é válido para agendar a análise no equipamento."""
        # Verifica se o equipamento já está em uso neste horário
        if any(equip == equipment for equip, _ in self.schedule[day][hour]):
            return False

        # Verifica se a análise já está sendo executada neste horário
        if any(anl == analysis for _, anl in self.schedule[day][hour]):
            return False

        # Verifica se o uso diário do equipamento não excede o limite
        if self.equipment_usage[day][equipment] >= self.equipment_limits[equipment]:
            return False

        return True

    def fitness(self):
        """Calcula o fitness do cronograma."""
        if not self.is_schedule_valid:
            return 0.0

        # Conta quantas análises foram totalmente agendadas
        fully_scheduled = 0
        analysis_equipment_count = defaultdict(int)

        for day in range(1, 8):
            for hour in range(8, 18):
                for equipment, analysis in self.schedule[day][hour]:
                    analysis_equipment_count[(analysis, equipment)] += 1

        # Verifica se cada análise tem todos os equipamentos agendados
        for analysis, equipment_list in self.analyses.items():
            all_scheduled = True
            for equipment in equipment_list:
                if analysis_equipment_count[(analysis, equipment)] == 0:
                    all_scheduled = False
                    break
            if all_scheduled:
                fully_scheduled += 1

        # Eficiência: relação entre o uso efetivo e o uso máximo disponível
        total_possible_usage = sum(self.equipment_limits.values()) * 7  # 7 dias
        actual_usage = sum(sum(usage.values()) for usage in self.equipment_usage.values())
        efficiency = actual_usage / total_possible_usage if total_possible_usage > 0 else 0

        # Compactação: menos dias usados implica um cronograma mais compacto
        total_days_used = len([day for day in range(1, 8) if any(self.schedule[day].values())])
        compactness = 1.0 / total_days_used if total_days_used > 0 else 0

        # Razão de completude
        completion_ratio = fully_scheduled / len(self.analyses)

        # Combinação dos critérios
        fitness = (completion_ratio * 0.7) + (efficiency * 0.2) + (compactness * 0.1)
        return fitness

    def mutacao(self):
        """Cria uma cópia mutada deste indivíduo."""
        mutated = copy.deepcopy(self)

        # Reseta o cronograma e uso dos equipamentos
        mutated.schedule = defaultdict(lambda: defaultdict(list))
        mutated.equipment_usage = defaultdict(lambda: defaultdict(int))

        # Lista de todos os pares (análise, equipamento)
        all_pairs = []
        for analysis, equipment_list in self.analyses.items():
            for equipment in equipment_list:
                all_pairs.append((analysis, equipment))

        random.shuffle(all_pairs)

        # Tenta reagendar cada par
        for analysis, equipment in all_pairs:
            all_slots = [(day, hour) for day in range(1, 8) for hour in range(8, 18)]
            random.shuffle(all_slots)

            scheduled = False
            for day, hour in all_slots:
                if mutated.is_valid_timeslot(analysis, day, hour, equipment):
                    mutated.schedule[day][hour].append((equipment, analysis))
                    mutated.equipment_usage[day][equipment] += 1
                    scheduled = True
                    break

            if not scheduled:
                mutated.is_schedule_valid = False
                return mutated

        mutated.is_schedule_valid = True
        return mutated

    def crossover(self, other):
        """Cria um filho combinando os cronogramas dos pais."""
        child = copy.deepcopy(self)

        # Reseta o cronograma e uso dos equipamentos
        child.schedule = defaultdict(lambda: defaultdict(list))
        child.equipment_usage = defaultdict(lambda: defaultdict(int))

        # Lista de todos os pares (análise, equipamento)
        all_pairs = []
        for analysis, equipment_list in self.analyses.items():
            for equipment in equipment_list:
                all_pairs.append((analysis, equipment))

        for analysis, equipment in all_pairs:
            parent1_slots = []
            parent2_slots = []

            for day in range(1, 8):
                for hour in range(8, 18):
                    if (equipment, analysis) in self.schedule[day][hour]:
                        parent1_slots.append((day, hour))
                    if (equipment, analysis) in other.schedule[day][hour]:
                        parent2_slots.append((day, hour))

            potential_slots = parent1_slots + parent2_slots
            random.shuffle(potential_slots)

            scheduled = False
            for day, hour in potential_slots:
                if child.is_valid_timeslot(analysis, day, hour, equipment):
                    child.schedule[day][hour].append((equipment, analysis))
                    child.equipment_usage[day][equipment] += 1
                    scheduled = True
                    break

            if not scheduled:
                all_slots = [(day, hour) for day in range(1, 8) for hour in range(8, 18)]
                random.shuffle(all_slots)
                for day, hour in all_slots:
                    if child.is_valid_timeslot(analysis, day, hour, equipment):
                        child.schedule[day][hour].append((equipment, analysis))
                        child.equipment_usage[day][equipment] += 1
                        scheduled = True
                        break

            if not scheduled:
                child.is_schedule_valid = False
                return child

        child.is_schedule_valid = True
        return child

    def imprime(self):
        return f"Fitness: {self.fitness():.4f}, Valid: {self.is_schedule_valid}"

    def display_schedule(self):
        print("Cronograma de Uso dos Equipamentos:")
        print("-" * 40)

        for day in range(1, 8):
            print(f"\nDia {day}:")
            print("---------")
            has_activity = False
            for hour in range(8, 18):
                if self.schedule[day][hour]:
                    has_activity = True
                    print(f"{hour}:00 - {hour+1}:00:", end=" ")
                    activities = []
                    for equipment, analysis in self.schedule[day][hour]:
                        activities.append(f"{equipment} - {analysis}")
                    print(", ".join(activities))
            if not has_activity:
                print("Sem atividades agendadas")

        print("\nResumo de Uso dos Equipamentos por Dia:")
        for day in range(1, 8):
            print(f"\nDia {day}:")
            for equipment in self.equipment_limits:
                usage = self.equipment_usage[day][equipment]
                print(f"  {equipment}: {usage} horas (Limite: {self.equipment_limits[equipment]} horas)")

    def display_analyses_schedule(self):
        print("\nCronograma por Análise:")
        print("-" * 40)
        analysis_schedule = defaultdict(list)

        for day in range(1, 8):
            for hour in range(8, 18):
                for equipment, analysis in self.schedule[day][hour]:
                    analysis_schedule[analysis].append((day, hour, equipment))

        for analysis in sorted(self.analyses.keys()):
            print(f"\n{analysis}:")
            if analysis in analysis_schedule:
                times = sorted(analysis_schedule[analysis], key=lambda x: (x[0], x[1]))
                for day, hour, equipment in times:
                    print(f"  Dia {day}, {hour}:00 - {hour+1}:00: {equipment}")
            else:
                print("  Não agendada")

class Populacao:
    def __init__(self, analyses, equipment_limits, tamanho_populacao=20):
        self.tamanho_populacao = tamanho_populacao
        self.analyses = analyses
        self.equipment_limits = equipment_limits
        self.populacao = []
        self.inicializacao()

    def inicializacao(self):
        for _ in range(self.tamanho_populacao):
            ind = Individuo(self.analyses, self.equipment_limits)
            self.populacao.append(ind)
        # Ordena pela fitness (maior primeiro)
        self.populacao.sort(key=self._fitness_populacao, reverse=True)

    def mutacao(self):
        nova_lista = []
        for individuo in self.populacao:
            nova_lista.append(individuo.mutacao())
        return nova_lista

    def crossover(self):
        nova_lista = []
        num_crossovers = self.tamanho_populacao

        for _ in range(num_crossovers):
            parent1 = self._tournament_selection(3)
            parent2 = self._tournament_selection(3)
            child = parent1.crossover(parent2)
            nova_lista.append(child)

        return nova_lista

    def _tournament_selection(self, tournament_size):
        tournament = random.sample(self.populacao, tournament_size)
        return max(tournament, key=self._fitness_populacao)

    def selecionar(self, populacao_mutada=None, populacao_crossover=None):
        candidates = list(self.populacao)
        if populacao_mutada:
            candidates.extend(populacao_mutada)
        if populacao_crossover:
            candidates.extend(populacao_crossover)
        # Seleciona os melhores
        nova_lista = sorted(candidates, key=self._fitness_populacao, reverse=True)
        self.populacao = nova_lista[:self.tamanho_populacao]

    def top_fitness(self):
        return self.top_individuo().fitness()

    def top_individuo(self):
        return self.populacao[0]

    def _fitness_populacao(self, individuo):
        return individuo.fitness()

class AlgoritmoGenetico:
    def __init__(self, populacao):
        self.populacao = populacao
        # Corrigido: inicializa o erro com base no melhor fitness da população
        self.erro = 1 - self.populacao.top_fitness()
        self.geracoes = 1

    def erro_final(self):
        return self.erro

    def qtd_geracoes(self):
        return self.geracoes

    def rodar(self, max_geracoes=1000, imprimir_em_geracaoes=100, erro_min=0.01):
        # Atualiza o erro com base no melhor indivíduo atual
        self.erro = 1 - self.populacao.top_fitness()
        print(f"Geração: {self.geracoes}, Erro: {round(self.erro, 3)}, {self.populacao.top_individuo().imprime()}")

        while self.geracoes < max_geracoes and self.erro > erro_min:
            populacao_mutada = self.populacao.mutacao()
            populacao_crossover = self.populacao.crossover()

            self.populacao.selecionar(populacao_mutada, populacao_crossover)
            fitness = self.populacao.top_fitness()
            # Atualiza o erro
            self.erro = 1 - fitness

            self.geracoes += 1
            if self.geracoes % imprimir_em_geracaoes == 0:
                print(f"Geração: {self.geracoes}, Erro: {round(self.erro, 3)}, {self.populacao.top_individuo().imprime()}")

        print(f"Geração: {self.geracoes}, Erro: {self.erro}, {self.populacao.top_individuo().imprime()}")
        return self.populacao.top_individuo()

In [6]:
# Dados de entrada
limites = {
    "Balança Analítica": 6,
    "Agitador Magnético": 4,
    "Cromatógrafo Líquido": 8,
    "Cromatógrafo Gasoso": 6,
    "Espectrofotômetro UV-VIS": 4,
    "Espectrômetro Infravermelho": 6,
    "Espectrômetro de Massa": 4,
    "Microscópio": 6
}

analises = {
    "Análise 1": ["Espectrofotômetro UV-VIS", "Cromatógrafo Gasoso"],
    "Análise 2": ["Cromatógrafo Líquido", "Espectrômetro Infravermelho"],
    "Análise 3": ["Microscópio", "Balança Analítica"],
    "Análise 4": ["Espectrômetro de Massa"],
    "Análise 5": ["Agitador Magnético", "Espectrômetro Infravermelho"],
    "Análise 6": ["Cromatógrafo Líquido", "Espectrofotômetro UV-VIS"],
    "Análise 7": ["Espectrofotômetro UV-VIS", "Microscópio"],
    "Análise 8": ["Cromatógrafo Gasoso"],
    "Análise 9": ["Espectrômetro Infravermelho", "Balança Analítica"],
    "Análise 10": ["Espectrômetro de Massa", "Cromatógrafo Gasoso"]
}

# Inicializa a população com tamanho desejado
populacao = Populacao(analises, limites, tamanho_populacao=50)
genetica = AlgoritmoGenetico(populacao)

# Executa o algoritmo genético
melhor_individuo = genetica.rodar(max_geracoes=500, imprimir_em_geracaoes=50, erro_min=0.001)

print("\n\nMelhor Solução Encontrada:")
print(f"Fitness: {melhor_individuo.fitness()}")
print(f"É um indivíduo Válido? {melhor_individuo.is_schedule_valid}")

melhor_individuo.display_schedule()
melhor_individuo.display_analyses_schedule()

Geração: 1, Erro: 0.268, Fitness: 0.7317, Valid: True
Geração: 50, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 100, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 150, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 200, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 250, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 300, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 350, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 400, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 450, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 500, Erro: 0.255, Fitness: 0.7450, Valid: True
Geração: 500, Erro: 0.2549783549783551, Fitness: 0.7450, Valid: True


Melhor Solução Encontrada:
Fitness: 0.7450216450216449
É um indivíduo Válido? True
Cronograma de Uso dos Equipamentos:
----------------------------------------

Dia 1:
---------
8:00 - 9:00: Balança Analítica - Análise 9
9:00 - 10:00: Espectrofotômetro UV-VIS - Análise 7
14:00 - 15:00: Cromatógrafo Gasoso - Análise